In [2]:
!pip3 install openpyxl

In [1]:
# import json 
import pandas as pd 
from pathlib import Path 
from glob import glob
import os

In [37]:
BASE_DIR = "./data/raw_input/" 

In [38]:
dfs = [] 
ignored_files = []
for filepath in glob(BASE_DIR + "/*"): 
    try: 
        tempdf = pd.read_json(filepath) 
        filename = Path(filepath).name
        tempdf['filename'] = filename
        dfs.append(tempdf) 
    except Exception as e: 
        ignored_files.append(filepath)

In [39]:
df = pd.concat(dfs, ignore_index=False).reset_index(drop=True)
df = df[df.tags.map(lambda x: len(x)) <= 1]
df['tags'] = df.tags.map(lambda x: x[0].strip() if x else "NA")

In [40]:
print(f"Number of rows: {df.shape[0]}")
print(f"Number of unique files: {pd.unique(df.filename).shape[0]}")
print(f"Number of unique tags: {pd.unique(df.tags).shape[0]}")

Number of rows: 13999
Number of unique files: 139
Number of unique tags: 39


In [41]:
all_tags = set(df.fillna("NA").tags.unique())

In [42]:
df[['tags']].groupby('tags').size().reset_index()

,tags,0
0,,239
1,Additional Liens,112
2,Amendments,166
3,Asset Disposition,68
4,Assignments,107
5,Collateral,1
6,Compliance Certificate,25
7,Consequences of Default,138
8,Default,2
9,Defaulted Interest,26


## Load Tag Mapping 

In [43]:
xl = pd.ExcelFile("./data/Updated-Tagging.xlsx")

In [44]:
xl.sheet_names

['Domain Tag List', 'AI Tag List']

In [45]:
tagging_df = pd.read_excel("./data/Updated-Tagging.xlsx", sheet_name = "AI Tag List")

In [46]:
standard_tags = set(tagging_df['Standard Tags'].unique())
non_standard_tags = set(tagging_df['Tusk Compare Fields'].unique())

In [47]:
nst_to_st = {t['Tusk Compare Fields'].strip() : t['Standard Tags'].strip() for t in tagging_df.to_dict(orient="records")}
for t in tagging_df.to_dict(orient="records"): 
    nst_to_st[t['Standard Tags'].strip()] = t['Standard Tags'].strip()

st_to_nst = {} 
for key, value in nst_to_st.items(): 
    st_to_nst[value] = key 

In [48]:
visited_st = {key: False for key in st_to_nst}
len(visited_st)

25

In [49]:
# print("Tags missing from the Updated Tagging:")
# for i, t in enumerate(all_tags - (standard_tags.union(non_standard_tags)), start=1):
#     print(i, t)
# print()
# print("Tags Missing from the data:")
# for i, t in enumerate((standard_tags.union(non_standard_tags)) - all_tags, start=1):
#     print(i, t, " - ",  nst_to_st[t])

In [50]:
tagging_df[['Standard Tags']].groupby("Standard Tags").size().reset_index()

,Standard Tags,0
0,Amendments,1
1,Amortization Schedule / Loan Repayment,1
2,Asset Disposition,1
3,Assignments,1
4,Compliance Certificate,1
5,Consequences of Default,1
6,Events of Default,1
7,Facilities / Instrument,1
8,Financial Covenant,1
9,Financial Statements,1


In [51]:
nst_to_st.update({
    "Facilities / Instruments": "Facilities / Instrument", 
    "Facilities": "Incremental Facilities", 
    "": "NA", 
    "Governing Law": "Governing Laws", 
    "Default" : "Event of Default",
    "Event of Default": "Event of Default",
    "Facilities" : "Incremental Facilities",
    "ESG Reporting Obligations": "Reporting Requirements", 
    "Subsidiary" : "NA",
    "Collateral" : "NA",
    "Subordination" : "NA",
    "Defaulted Interest" : "Pricing / Interest", 
    "Prepayments" : "Prepayment / Redemption",
    "Interest Payment Dates":  "Pricing / Interest",
    "NA": "NA",
    "Use of Proceeds": "NA", 
    "Other Reporting Requirements": "Reporting Requirements", 
    "Interest / Pricing": "Pricing / Interest",

    
})

In [52]:
for tag in all_tags: # all tags from dataset 
    if tag in nst_to_st:
        visited_st[nst_to_st[tag]] = True
    else:
        print(tag)

In [53]:
for key, visited in visited_st.items(): 
    if not visited: 
        print(key)

Other Reporting Requirements


In [54]:
print("Total Size of Updated Tagging", tagging_df.shape)

Total Size of Updated Tagging (25, 3)


In [55]:
df[['tags']].groupby('tags').size().reset_index().sort_values(by=0, ascending=False)

,tags,0
24,NA,11635
0,,239
13,Facilities / Instrument,173
2,Amendments,166
7,Consequences of Default,138
18,Governing Laws,114
22,Loan Repayment,113
1,Additional Liens,112
21,Interest Rate,112
4,Assignments,107


In [56]:
print(f"Number of files with empty section content: {pd.unique(df[df['section_content'].isna()].filename).shape[0]}")
print(f"Number of files with empty section and tag present content: {pd.unique(df[(df['section_content'].isna()) & (df['tags'].notna()) ].filename).shape[0]}")
print(f"Number of records where section is empty: {df[(df['section_content'].isna())].shape[0]}")

Number of files with empty section content: 50
Number of files with empty section and tag present content: 50
Number of records where section is empty: 504


In [57]:
df['section_content'] = df['section_content'].fillna("")
df['tags'] = df['tags'].fillna("NA")
df['title'] = df['title'].fillna("") 
df['category'] = df['category'].fillna("")

In [58]:
df['word_count'] = df['section_content'].fillna("").apply(lambda x : len(x.split()))

# Filter section content with small text

In [59]:
df = df[(df.word_count > 20)]
df = df.fillna("NA") 

In [60]:
# df['section_content'] = df['title'].apply(lambda x: f"Title is {x}") + "\n" + df['category'].apply(lambda x: f"Category is {x}") + "\n" +  df['section_content']
df['section_content'] = df['title'].apply(lambda x: f"Title is {x}") + "\n" +  df['section_content']

In [61]:
df['standard_tags'] = df.tags.map(lambda x: nst_to_st[x])

In [62]:
df.to_json("./data/all-data-raw-standard-tags.json")

In [34]:
df.word_count.describe()

count    13032.000000
mean       472.137431
std        687.562411
min         21.000000
25%        114.000000
50%        245.000000
75%        543.000000
max       9381.000000
Name: word_count, dtype: float64

# Preprocessing 

In [35]:
df = pd.read_json("./data/all-data-raw-standard-tags.json")

In [36]:
df = df.drop_duplicates('section_content') 

## Select content only form shortlisted files

In [37]:
file_analysis_df = pd.read_excel("./data/File-Analysis.xlsx")

In [38]:
good_files = set(file_analysis_df.File.to_list()[:100])

In [39]:
df['tags'] = df['tags'].str.replace("Events of Default", "Event of Default")

In [40]:
# df = df[df.filename.str.contains("Agreement", case=False)]

In [41]:
df = df.drop_duplicates('section_content', keep=False)

In [42]:
df = df[df.filename.map(lambda x: x in good_files)]

In [43]:
df['standard_tags'] = df.tags.map(lambda x: nst_to_st[x])

In [44]:
df[['standard_tags']].groupby('standard_tags').size().reset_index().sort_values(by=0)

,standard_tags,0
12,Interest / Pricing,12
4,Compliance Certificate,13
13,Mandatory Prepayments / Redemption,14
9,Financial Statements,23
22,Restricted Investments,23
15,Optional Prepayment / Redemption,28
23,Restricted Payments,42
11,Incremental Facilities,42
16,Permitted Indebtedness,45
6,Event of Default,47


In [45]:
df[['tags']].groupby('tags').size().reset_index().sort_values(by=0)

,tags,0
6,Default,1
15,Governing Law,1
12,Facilities / Instruments,1
28,Prepayments,1
35,Use of Proceeds,1
33,Subsidiary,1
10,Facilities,2
32,Subordination,2
24,Other Reporting Requirements,5
18,Interest Payment Dates,5


In [46]:
df.word_count.describe()

count    8282.000000
mean      497.042864
std       686.534330
min        21.000000
25%       121.000000
50%       263.000000
75%       584.000000
max      9381.000000
Name: word_count, dtype: float64

In [47]:
df =  df.drop(columns = 'tags')

In [48]:
df = df.rename(columns = {'standard_tags': 'tags'})
df.head()

,title,category,checklists,section_content,filename,word_count,tags
0,Computation of Time Periods,FYI,[],Title is Computation of Time Periods\nSection ...,"Abbvie - 6,000,000,000 TERM LOAN CREDIT AGREEM...",51,NA
1,Accounting Terms,FYI,[],Title is Accounting Terms\nSection 1.03. Accou...,"Abbvie - 6,000,000,000 TERM LOAN CREDIT AGREEM...",430,NA
2,Terms Generally,FYI,[],Title is Terms Generally\nSection 1.04. Terms ...,"Abbvie - 6,000,000,000 TERM LOAN CREDIT AGREEM...",255,NA
3,Divisions,FYI,[],Title is Divisions\nSection 1.05. Divisions. F...,"Abbvie - 6,000,000,000 TERM LOAN CREDIT AGREEM...",106,NA
4,The Advances,FYI,[],Title is The Advances\nSection 2.01. The Advan...,"Abbvie - 6,000,000,000 TERM LOAN CREDIT AGREEM...",571,Facilities / Instrument


In [77]:
print(df.shape)

(8282, 8)


In [49]:
for t in df[['tags']].groupby('tags').size().reset_index().sort_values(by=0).itertuples(): 
    print(t.tags,":", t._2)

Interest / Pricing : 12
Compliance Certificate : 13
Mandatory Prepayments / Redemption : 14
Financial Statements : 23
Restricted Investments : 23
Optional Prepayment / Redemption : 28
Restricted Payments : 42
Incremental Facilities : 42
Permitted Indebtedness : 45
Event of Default : 47
Consequences of Default : 47
Asset Disposition : 48
Transactions with Affiliates : 48
Prepayment / Redemption : 57
Waterfall of Proceeds : 65
Governing Laws : 73
Reporting Requirements : 73
Financial Covenant : 77
Premium and Fees : 80
Permitted Liens : 81
Assignments : 82
Amortization Schedule / Loan Repayment : 83
Pricing / Interest : 89
Amendments : 105
Facilities / Instrument : 135
NA : 6850


In [50]:
df.word_count.describe()

count    8282.000000
mean      497.042864
std       686.534330
min        21.000000
25%       121.000000
50%       263.000000
75%       584.000000
max      9381.000000
Name: word_count, dtype: float64

In [51]:
df[df.tags == "NA"][['tags', 'title', 'category']].groupby('tags').agg(set)

,title,category
tags,,
NA,"{, Telephonic Notices, Delayed Draw Term Loan ...","{Affirmative Covenants, Negative Covenants, Ev..."


In [52]:
agg_df = df[['tags', 'title' ]].groupby([ 'tags', 'title']).size().reset_index().sort_values(by=0).rename(columns={0:'freq'})

In [53]:
tag_title_filter = {t.tags+""+t.title for t in agg_df[agg_df.tags == "NA"][agg_df.freq <=3].itertuples()}

/tmp/ipykernel_67/497309628.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  tag_title_filter = {t.tags+""+t.title for t in agg_df[agg_df.tags == "NA"][agg_df.freq <=3].itertuples()}


In [54]:
len(tag_title_filter)

1807

In [55]:
df['tag_title'] = df.tags+df.title

In [56]:
new_df = pd.concat([df[df.tags!="NA"], df[df.tag_title.isin( tag_title_filter)]])

In [57]:
new_df.word_count.describe()

count    3948.000000
mean      588.744934
std       798.959793
min        21.000000
25%       127.000000
50%       292.000000
75%       692.500000
max      9381.000000
Name: word_count, dtype: float64

In [68]:
sum(new_df[['tags']].groupby('tags').size().reset_index().sort_values(by=0).rename(columns={0:"freq"}).freq.values)

3948

# Save Data cleaned for NA tags

In [63]:
new_df.to_json("./data/cleaned_tags_data_na_tag_v2_lt3.json", orient='records')

In [72]:
print(new_df.shape)

(3948, 8)


In [73]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [74]:
unique_filenames = df.filename.unique()

# TODO  Check correct data frame for train test split

In [75]:
# Split filenames into train and test sets
train_files, test_files = train_test_split(unique_filenames, test_size=0.2, random_state=42)

# Create train and test datasets based on the split filenames
train_df = new_df[new_df['filename'].isin(train_files)].reset_index() 
test_df = df[df['filename'].isin(test_files)].reset_index()
del train_df['index'] 
del test_df['index'] 
train_df.shape, test_df.shape

((6789, 8), (1493, 8))

In [76]:
selected_columns = ["section_content", "tags"]

In [101]:
train_tags = set(train_df.tags.unique())
test_tags = set(test_df.tags.unique())

In [102]:
print(test_tags - train_tags )
assert not test_tags - train_tags 

set()


In [103]:
OUTPUT_DIR = "./data/dataset/v8"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [104]:
train_df.head()

,title,category,checklists,section_content,filename,word_count,tags,tag_title
0,Terms Generally,FYI,[],Title is Terms Generally\nSection 1.2. Terms G...,"Paramount Global - $3,500,000,000 AMENDED AND ...",925,NA,NATerms Generally
1,Currency Equivalents,FYI,[],Title is Currency Equivalents\nSection 1.3. Cu...,"Paramount Global - $3,500,000,000 AMENDED AND ...",42,NA,NACurrency Equivalents
2,Commitments,Event Driven,[],Title is Commitments\nSection 2.1. Commitments...,"Paramount Global - $3,500,000,000 AMENDED AND ...",205,Facilities / Instrument,Facilities / InstrumentCommitments
3,Revolving Credit Loans; Competitive Loans,Event Driven,[],Title is Revolving Credit Loans; Competitive L...,"Paramount Global - $3,500,000,000 AMENDED AND ...",422,Facilities / Instrument,Facilities / InstrumentRevolving Credit Loans;...
4,Competitive Bid Procedure,Event Driven,[],Title is Competitive Bid Procedure\nSection 2....,"Paramount Global - $3,500,000,000 AMENDED AND ...",1879,NA,NACompetitive Bid Procedure


In [105]:
print(train_df.shape)
print(test_df.shape)

(6789, 8)
(1493, 8)


In [106]:
train_df.to_json(os.path.join(OUTPUT_DIR, "train.json")) 
test_df.to_json(os.path.join(OUTPUT_DIR, "test.json"))

# Integrate Domain Correction 

In [7]:
import pandas as pd

In [8]:
xl = pd.ExcelFile("./data/Predictions_v13_small_with_sec_numbering_cleaned.xlsx")

In [9]:
xl.sheet_names

['Sheet1']

In [35]:
updated_df = pd.read_excel("./data/Predictions_v13_small_with_sec_numbering_cleaned.xlsx")

In [36]:
updated_df.columns

Index(['filename', 'title', 'section_number', 'section_content',
       'Original_Tag', 'Predicted_Tag', 'match', 'Domain', 'Corrected',
       'Unnamed: 9'],
      dtype='object')

In [37]:
updated_df = updated_df[['filename', 'title', 'section_number', 'section_content',
       'Original_Tag', 'Predicted_Tag', 'match', 'Domain']]

In [38]:
updated_df = updated_df.fillna("NA")

In [39]:
updated_df['Domain'] = updated_df.Domain.str.strip()

In [40]:
domain_tags = set(updated_df.Domain.values)
all_tags = set(updated_df.Original_Tag.values)
print(len(domain_tags))
print(len(all_tags))
print("\nCorrection Tags:")
print("\n".join(domain_tags - all_tags))

29
26

Correction Tags:
Permitted Indebtedness, Permitted Liens, Asset Disposition
Compliance Certificate, Other Reporting Requirements
Facilities / Instrument, Incremental Facilities
Other Reporting Requirements


In [41]:
print("\n".join(sorted(all_tags)))

Amendments
Amortization Schedule / Loan Repayment
Asset Disposition
Assignments
Compliance Certificate
Consequences of Default
Event of Default
Facilities / Instrument
Financial Covenant
Financial Statements
Governing Laws
Incremental Facilities
Interest / Pricing
Mandatory Prepayments / Redemption
NA
Optional Prepayment / Redemption
Permitted Indebtedness
Permitted Liens
Premium and Fees
Prepayment / Redemption
Pricing / Interest
Reporting Requirements
Restricted Investments
Restricted Payments
Transactions with Affiliates
Waterfall of Proceeds


In [43]:
nst_to_st = {
    "Interest / Pricing": "Pricing / Interest",
    "Other Reporting Requirements": "Reporting Requirements"
}

In [44]:
def to_standard_tag(x): 
    if x in nst_to_st: 
        return nst_to_st.get(x)
    return x
        
    

In [45]:
updated_df['Domain'] = updated_df.Domain.apply(to_standard_tag)

In [52]:
updated_df = updated_df[updated_df.Domain.map(lambda x: x in all_tags)]

In [53]:
domain_tags = set(updated_df.Domain.values)
all_tags = set(updated_df.Original_Tag.values)
print(len(domain_tags))
print(len(all_tags))
print("\nCorrection Tags:")
print("\n".join(domain_tags - all_tags))

25
26

Correction Tags:



In [54]:
print("\n".join(sorted(domain_tags)))

Amendments
Amortization Schedule / Loan Repayment
Asset Disposition
Assignments
Compliance Certificate
Consequences of Default
Event of Default
Facilities / Instrument
Financial Covenant
Financial Statements
Governing Laws
Incremental Facilities
Mandatory Prepayments / Redemption
NA
Optional Prepayment / Redemption
Permitted Indebtedness
Permitted Liens
Premium and Fees
Prepayment / Redemption
Pricing / Interest
Reporting Requirements
Restricted Investments
Restricted Payments
Transactions with Affiliates
Waterfall of Proceeds


In [57]:
updated_df['tags'] = updated_df['Domain']

In [59]:
# updated_df.to_json("./data/dataset/v9/train.json")
updated_df.to_excel("./data/v9_train.xlsx", index=False)

In [43]:
"""
1. Domain NA Tags 
2. Multiple tags 
3. Blank means ok. 
Optional Prepayment -> Optional Prepayment / Redemption
"""

'\n1. Domain NA Tags \n2. Multiple tags \n\nOptional Prepayment -> Optional Prepayment / Redemption\n'